# loading data from MySQL

In [ ]:
!pip install pymysql 

Set the database connection. in this example we are using a public mysql database called Rfam (https://rfam.readthedocs.io/en/latest/database.html)<br>
Then run the sql and keep the result into the dataframe

In [2]:
import os
import pymysql
import pandas as pd

host = os.getenv('mysql-rfam-public.ebi.ac.uk')
port = os.getenv('4497')
user = os.getenv('rfamro')
password = os.getenv('')
database = os.getenv('Rfam ')

conn = pymysql.connect(
    host="mysql-rfam-public.ebi.ac.uk",
    port=int(4497),
    user="rfamro",
    passwd=password,
    db="Rfam",
    charset='utf8mb4')

df = pd.read_sql_query("select rfam_acc,rfam_id,auto_wiki,description,author,seed_source  FROM family",
    conn) 


#df.tail(10)
#df.describe()
#df.info()



In [3]:
import pandas as pd
import v3io_frames as v3f
client = v3f.Client('http://v3io-framesd:8080')
tablename = 'examples/family'

Ingesting data into the database using NoSQL API

In [4]:
out = client.write('kv', tablename, df)

## Remove Data

In [5]:
!rm -rf /v3io/bigdata/examples/family